In [1]:
!pip install backtesting
import scipy
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pandas_ta as ta
import yfinance as yahooFinance
import datetime
!pip install pandas_ta
import pandas_ta as ta

In [2]:
GetFacebookInformation = yahooFinance.Ticker("META")
startDate = datetime.datetime(2012, 5, 31)

# endDate , as per our convenience we can modify
endDate = datetime.datetime(2024, 4, 30)
data = GetFacebookInformation.history(start=startDate,
                                     end=endDate)